In [ ]:
from luminoso_api import LuminosoClient
from pack64 import unpack64
from tableau_export import pull_lumi_data

import numpy as np

In [ ]:
client, docs, topics, terms, subsets, drivers, skt, themes = pull_lumi_data('d53m338v', 'pr3n28bw')

In [ ]:
print(len(unpack64(topics[0]['vector'])))

In [ ]:
print(len(unpack64(docs[0]['vector'])))

In [ ]:
np.dot(unpack64(topics[0]['vector']), unpack64(docs[0]['vector']))

In [ ]:
len(terms)

In [ ]:
topics[0]

In [ ]:
%run tableau_export.py d53m338v pr3n28bw -t 150

In [ ]:
client = LuminosoClient.connect()

In [ ]:
client = client.change_path('prs8wfq6')

In [ ]:
docs = client.get('docs')

In [ ]:
docs[0]

In [1]:
%run tableau_export.py demo qdwr5

Extracting Lumi data...
14999
14999
14999
14997
14999
14996
14999
14997
3534
14995
14133
3775
345
2855
651
14995
199
723
14998
5595
5870
3838
14997
14996
14997
14997
14999
14996
Creating doc table...


/usr/local/lib/python3.6/site-packages/se_code-0.1-py3.6.egg/se_code/fuzzy_logic.py:38: RuntimeWarning: invalid value encountered in double_scalars


Writing to file doc_table.csv.
Writing to file xref_table.csv.
Creating themes table...
Writing to file themes_table.csv.
Creating subset key terms table...
Writing to file skt_table.csv.
Writing to file drivers_table.csv.
Writing to file trends_table.csv.
Writing to file trendingterms_table.csv.
